In [1]:
# 导入模块

%matplotlib notebook


import numpy as np
import matplotlib.pyplot as plt
from obspy import UTCDateTime, read, read_inventory, read_events
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.header import FDSNNoDataException
from datetime import datetime, timedelta, timezone

import pandas as pd
from scipy.interpolate import interp1d
import os
import shutil

from time import time
import pickle

In [2]:
# 定义数据所在文件夹

cwd = os.getcwd()
data_dir = os.path.join(cwd, "data_DA")
raw_waveform_dir = os.path.join(data_dir, "waveforms_raw")
processed_waveform_dir = os.path.join(data_dir, "waveforms_proc_broadband")
print(cwd)
print(data_dir)
print(raw_waveform_dir)
print(processed_waveform_dir)

F:\code\GNN_seismic_location\version1
F:\code\GNN_seismic_location\version1\data_DA
F:\code\GNN_seismic_location\version1\data_DA\waveforms_raw
F:\code\GNN_seismic_location\version1\data_DA\waveforms_proc_broadband


In [3]:
# 移除旧文件夹并创建新文件夹
if os.path.isdir(data_dir):
    print("data directory exists")
else:
    print("data directory does not exist")
    os.mkdir(data_dir)

if os.path.isdir(raw_waveform_dir):
    print("Removing old raw waveform directory...")
    shutil.rmtree(raw_waveform_dir)
os.mkdir(raw_waveform_dir)

if os.path.isdir(processed_waveform_dir):
    print("Removing old processed waveform directory...")
    shutil.rmtree(processed_waveform_dir)
os.mkdir(processed_waveform_dir)

data directory exists
Removing old raw waveform directory...
Removing old processed waveform directory...


# 台站清单与事件目录

首先，我们定义数据采集的参数。我们选用南加州地震网络（CI）2000–2020 年的目录，并选取所有纬度在 32°–36°、经度在 −120°–−116° 范围内记录的地震台站和事件。

In [4]:
# 参数

client = Client("IRIS")
network = "CI"  # 南加州地震台网

inventory_filename = "inventory_DA.xml"
catalogue_filename = "catalogue_DA.file"

starttime = UTCDateTime("2000-01-01T00:00:00")
endtime = UTCDateTime("2020-01-01T00:00:00")

station_settings = {
    "network": "CI",
    "minlatitude": 32,
    "maxlatitude": 36,
    "minlongitude": -120,
    "maxlongitude": -116,
    "startbefore": endtime,
    "endafter": starttime,
    "level": "response",
    "filename": inventory_filename,
    "format": "xml",
    "channel": "BHN,BHE,BHZ",
}

event_settings = {
    "minlatitude": 32,
    "maxlatitude": 36,
    "minlongitude": -120,
    "maxlongitude": -116,
    "starttime": starttime,
    "endtime": endtime,
    "minmagnitude": 3.0,
    "filename": catalogue_filename,
}

In [5]:
# 获取地震台站（可能要几分钟）

client.get_stations(**station_settings)

我们创建一个包含若干有用台站元数据的 Pandas 数据框：

In [6]:
# 读取台站列表
stations = read_inventory(inventory_filename).select(network=network)[0]
# 获取台站代码（按字母排序）
station_codes = np.sort([station.code for station in stations])
# 定义 DataFrame
# station_df = pd.DataFrame(columns=("code", "lat", "lon", "start_date", "end_date"))
# 
# # 遍历排序后的台站代码
# for station_code in station_codes:
#     # 选择该台站的数据
#     station = stations.select(station=station_code)[0]
#     # 将台站元数据追加到 DataFrame
#     station_df = station_df.append(
#         {
#             "code":       station_code,
#             "lat":        station.latitude,
#             "lon":        station.longitude,
#             "start_date": station.start_date,
#             "end_date":   station.end_date,
#         }, ignore_index=True
#     )

# 用一个列表先存所有行字典
rows = []
for station_code in station_codes:
    station = stations.select(station=station_code)[0]
    rows.append({
        "code":       station_code,
        "lat":        station.latitude,
        "lon":        station.longitude,
        "start_date": station.start_date,
        "end_date":   station.end_date,
    })
# 最后一次性把列表变成 DataFrame
station_df = pd.DataFrame(rows, columns=["code","lat","lon","start_date","end_date"])

# 将 DataFrame 保存为 CSV 文件
station_df.to_csv(os.path.join(data_dir, "stations.csv"), index=False)

作为一个快速的合理性检查，我们绘制台站的位置：

In [7]:
# 打印台站数量
print(len(station_df))
# 绘制台站位置
pl = stations.plot(projection="local", resolution="i", show=True)
plt.show()

213


<IPython.core.display.Javascript object>

接下来，我们下载 SCSN 事件目录：
“SCSN 事件” 指的是由 Southern California Seismic Network（南加州地震网） 记录和编录的地震事件。

In [9]:
client.get_events(**event_settings)

In [10]:
# 从文件中读取事件目录
catalogue = read_events(catalogue_filename)
print(catalogue)

2542 Event(s) in Catalog:
2019-12-28T17:23:32.020000Z | +33.927, -117.876 | 3.2  Ml
2019-12-28T08:43:29.490000Z | +35.615, -117.420 | 3.15 Ml
...
2000-01-02T17:58:32.230000Z | +34.309, -116.081 | 3.2  ML
2000-01-01T03:58:38.570000Z | +34.835, -116.288 | 3.1  ML
To see all events call 'print(CatalogObject.__str__(print_all=True))'


同样地，我们绘制一些事件，以检查一切是否按预期进行：

In [11]:
catalogue.plot(projection="local", resolution="i", show=True)
plt.show()

<IPython.core.display.Javascript object>

# 下载原始波形

在下面的单元格中，我们下载波形并为事件发生时处于运行状态的台站创建一个查找表。在循环遍历目录中的所有事件时，我们选择“首选”起源时间、位置和震级，并检查哪些台站在起源时间前后处于运行状态。然后，我们为每个运行台站下载起源时间前60秒到起源时间后最长 180 秒的三分量波形。

**注意 1**：所谓“运行”台站并不意味着它们一定录得了数据。我们会在后续步骤中进行额外的质量检查。

**注意 2**：即使在网络速度很快的情况下，下载所有波形也可能需要几个小时。

In [21]:
import threading
wait = threading.Event().wait

# 初始化事件目录 DataFrame
catalogue_df = pd.DataFrame(columns=("time", "lat", "lon", "depth", "mag"))
# 初始化台站查找表
catalogue_station_lookup = {}

t0 = time()
ETA = 0
for i in range (10):
    print('___')

# 遍历 SCSN 事件目录中的事件
for i, event in enumerate(catalogue):
    
    # 根据平均处理时间计算 ETA
    running_time = time() - t0
    events_per_sec = (i + 1) / running_time
    events_to_do = len(catalogue) - i
    ETA = events_to_do / events_per_sec
    
    print("Fetching event %d / %d [ETA: %.d s]" % (i+1, len(catalogue), ETA))
    
    # 获取事件的首选起源信息和震级信息
    pref_origin = event.preferred_origin_id
    pref_magnitude = event.preferred_magnitude_id
    origins = event.origins
    magnitudes = event.magnitudes
    
    # 选择首选起源
    for origin in origins:
        if origin.resource_id == pref_origin:
            break
    
    # 选择首选震级
    for magnitude in magnitudes:
        if magnitude.resource_id == pref_magnitude:
            break
    
    # 将事件信息追加到 DataFrame
    # catalogue_df = catalogue_df.append(
    #     {"time": origin.time, "lat": origin.latitude, "lon": origin.longitude, "depth": origin.depth, "mag": magnitude.mag}, 
    #     ignore_index=True
    # )
    idx = len(catalogue_df)  # 新行的索引
    catalogue_df.loc[idx] = [
        origin.time,
        origin.latitude,
        origin.longitude,
        origin.depth,
        magnitude.mag
    ]
    
    # 波形输出文件路径
    waveform_file = os.path.join(raw_waveform_dir, "%d.sac" % i)
    
    # 定义波形起止时间
    start_trace = origin.time - timedelta(seconds=60)
    end_trace = origin.time + timedelta(seconds=180)
    
    # 检查哪些台站在事件时刻处于运行状态
    # 台站启用时间早于波形开始时间
    station_inds = (station_df["start_date"] < start_trace)
    # 台站退役时间晚于波形结束时间或仍在运行
    station_inds = station_inds & ((station_df["end_date"] > end_trace) | station_df["end_date"].isnull())
    # 获取运行台站代码并排序
    operational_stations = np.sort(station_df["code"][station_inds].values)
    # 将运行台站添加到查找表
    catalogue_station_lookup[i] = operational_stations
    
    # 构造批量下载波形的查询列表
    bulk_query = [(network, station, "*", "BH*", start_trace, end_trace) for station in operational_stations]
    
    print("Fetching data for %d stations" % len(operational_stations))
    
    # 请求波形数据并写入文件
    try:
        client.get_waveforms_bulk(bulk_query, filename=waveform_file)
    # 如果 FDSN 无数据，则跳过
    except FDSNNoDataException:
        print("No data...")
        continue
    # wait(1.0) 

# 将事件目录保存为 CSV
catalogue_df.to_csv(os.path.join(data_dir, "catalogue.csv"))

# 将查找表保存为二进制对象
lookup_file = os.path.join(data_dir, "catalogue_station_lookup.pickle")
with open(lookup_file, "wb") as f:
    pickle.dump(catalogue_station_lookup, f, protocol=pickle.HIGHEST_PROTOCOL)


___
___
___
___
___
___
___
___
___
___
Fetching event 1 / 2542 [ETA: 39 s]
Fetching data for 188 stations
Fetching event 2 / 2542 [ETA: 27 s]
Fetching data for 188 stations
Fetching event 3 / 2542 [ETA: 23 s]
Fetching data for 188 stations
Fetching event 4 / 2542 [ETA: 19 s]
Fetching data for 188 stations
Fetching event 5 / 2542 [ETA: 17 s]
Fetching data for 188 stations
Fetching event 6 / 2542 [ETA: 14 s]
Fetching data for 188 stations
Fetching event 7 / 2542 [ETA: 13 s]
Fetching data for 188 stations
Fetching event 8 / 2542 [ETA: 12 s]
Fetching data for 188 stations
Fetching event 9 / 2542 [ETA: 11 s]
Fetching data for 188 stations
Fetching event 10 / 2542 [ETA: 10 s]
Fetching data for 188 stations
Fetching event 11 / 2542 [ETA: 10 s]
Fetching data for 188 stations
Fetching event 12 / 2542 [ETA: 9 s]
Fetching data for 188 stations
Fetching event 13 / 2542 [ETA: 9 s]
Fetching data for 188 stations
Fetching event 14 / 2542 [ETA: 8 s]
Fetching data for 188 stations
Fetching event 15 / 


# 处理波形数据

 在下载原始波形后，我们对其进行预处理，以作为深度学习模型的输入。首先检查每个台站的响应是否可用。
然后对波形在 0.1 到 8 Hz 之间进行滤波，并将数据插值到一个采样率约为 20 Hz 的统一时间基准上。

 **注意**：这一步同样需要数小时

In [22]:
from obspy.core.util.obspy_types import ObsPyException

# 读取事件目录
catalogue_df = pd.read_csv(os.path.join(data_dir, "catalogue.csv"))
# 将时间转换为时间戳
catalogue_df["time"] = pd.to_datetime(catalogue_df["time"])

Ntrace = 3     # 波形分量数量
Nt = 4096     # 时间采样点数量
scale = 1e5    # 波形振幅缩放因子

t0_ETA = time()
ETA = 0

# 初始化事件-台站查找表
catalogue_station_lookup_final = {}

# 遍历目录中的所有事件
for i, event in catalogue_df.iterrows():
    
    # 根据平均处理时间计算剩余时间 ETA
    running_time = time() - t0_ETA
    events_per_sec = (i + 1) / running_time
    events_to_do = len(catalogue) - i
    ETA = events_to_do / events_per_sec
    
    print("Processing event %d / %d [ETA: %d s]" % (i+1, len(catalogue), ETA))
    
    # 定义原始波形文件路径
    event_file = os.path.join(raw_waveform_dir, "%d.sac" % i)
    # 如果文件不存在（可能因上一步 FDSN 错误），则跳过
    if not os.path.isfile(event_file):
        print("Event %d not found" % (i + 1))
        continue
    
    # 获取该事件的运行台站列表
    operational_stations = catalogue_station_lookup[i]
    
    # 运行台站数量
    Nstations = len(operational_stations)
    
    # 从文件读取波形数据
    data = read(event_file)
    
    # 创建统一的时间基准：起源后 1~101 秒
    starttime = event["time"] - timedelta(seconds=40)
    endtime = starttime + timedelta(seconds=200)
    time_base = pd.date_range(starttime, endtime, periods=Nt).to_pydatetime()
    # 将时间基准转换为 UTC 时间戳（含夏令时等信息）
    timestamp_base = np.array([dt.replace(tzinfo=timezone.utc).timestamp() for dt in time_base])
    # 使时间基准相对于起始时间
    t0 = timestamp_base.min()
    t_int = timestamp_base - t0
    
    # 初始化处理后波形数据缓冲区（默认 NaN，便于后续剔除无效记录）
    proc_data = np.zeros((Nstations, Nt, 3)) * np.nan
    
    # 遍历运行台站列表
    for k, station in enumerate(operational_stations):
        # 选择当前台站的数据流
        stream = data.select(station=station)
        # 检查是否恰好包含 3 个分量
        if len(stream) == Ntrace:
            # 对波形去直流分量（减去均值）
            stream.detrend("simple") 
            # 尝试去除台站响应（不总是可行）
            try:
                stream.remove_response(inventory=stations, zero_mean=True)
            # 如果缺少响应信息
            except ValueError:
                print("Response could not be removed")
                continue
            # ObsPy 有时会抛出异常
            except ObsPyException as e:
                print("Weird Obspy exception")
                continue
            # 处理成功后，对波形进行 0.1~8 Hz 带通滤波
            stream.filter("bandpass", freqmin=0.1, freqmax=8.0, zerophase=True)
            # 遍历各分量
            for n in range(Ntrace):
                # 提取单个分量数据
                trace = stream[n]
                tr_data = trace.data
                # 再次去均值（保险起见）
                tr_data -= tr_data.mean()
                # 进行振幅缩放
                tr_data *= scale
                # 获取分量的时间戳
                t = trace.times(type="timestamp") - t0
                # 插值到统一时间基准并存入缓冲区
                proc_data[k, :, n] = interp1d(
                    t, tr_data,
                    fill_value=0,
                    bounds_error=False,
                    assume_sorted=True
                )(t_int)
    
    # 筛选出有效台站（即对应行非 NaN）
    ok_inds = np.isfinite(proc_data[:, 0, 0])
    # 将有效台站添加到最终查找表
    catalogue_station_lookup_final[i] = operational_stations[ok_inds]
    # 定义处理后波形文件路径
    proc_data_file = os.path.join(processed_waveform_dir, "%d.npy" % i)
    # 保存处理后波形数据
    np.save(proc_data_file, proc_data[ok_inds])

# 将最终查找表写入磁盘
lookup_file = os.path.join(data_dir, "catalogue_station_lookup_final.pickle")
with open(lookup_file, "wb") as f:
    pickle.dump(catalogue_station_lookup_final, f, protocol=pickle.HIGHEST_PROTOCOL)


Processing event 1 / 2542 [ETA: 5 s]
Processing event 2 / 2542 [ETA: 2776 s]
Processing event 3 / 2542 [ETA: 2643 s]
Processing event 4 / 2542 [ETA: 2565 s]
Processing event 5 / 2542 [ETA: 2519 s]
Processing event 6 / 2542 [ETA: 2478 s]
Processing event 7 / 2542 [ETA: 2438 s]
Processing event 8 / 2542 [ETA: 2408 s]
Processing event 9 / 2542 [ETA: 2385 s]
Processing event 10 / 2542 [ETA: 2367 s]
Processing event 11 / 2542 [ETA: 2354 s]
Processing event 12 / 2542 [ETA: 2343 s]
Processing event 13 / 2542 [ETA: 2336 s]
Processing event 14 / 2542 [ETA: 2333 s]
Processing event 15 / 2542 [ETA: 2325 s]
Processing event 16 / 2542 [ETA: 2317 s]
Processing event 17 / 2542 [ETA: 2309 s]
Processing event 18 / 2542 [ETA: 2302 s]
Processing event 19 / 2542 [ETA: 2297 s]
Processing event 20 / 2542 [ETA: 2291 s]
Processing event 21 / 2542 [ETA: 2293 s]
Processing event 22 / 2542 [ETA: 2290 s]
Processing event 23 / 2542 [ETA: 2284 s]
Processing event 24 / 2542 [ETA: 2277 s]
Processing event 25 / 2542 [

绘制震级最大的事件的波形，以快速检查：

In [23]:
# 找到震级最大的事件索引
event_id = catalogue_df["mag"].idxmax()
# 打印该事件的索引和对应的行数据
print(event_id, catalogue_df.loc[event_id])
# 从磁盘加载该事件的处理后波形数据（NumPy 格式）
event_data = np.load(os.path.join(processed_waveform_dir, "%d.npy" % event_id))
# 创建一个 1 行 3 列的图，三个子图共用 x、y 轴
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(9, 9), sharex="all", sharey="all")

# 逐站绘制波形（按分量分别绘在三个子图中）
for i, trace in enumerate(event_data):
    for j in range(Ntrace):
        # 如果该站该分量全是零（表示无数据），则用红色
        if np.abs(trace[:, j].sum() == 0):
            axes[j].plot(trace[:, j] + i * event_data.std(), c="r", lw=1)
        # 否则正常数据用黑色绘制
        else:
            axes[j].plot(trace[:, j] + i * event_data.std(), c="k", lw=1)

# 调整子图布局并显示
plt.tight_layout()
plt.show()

726 Unnamed: 0                                 726
time          2019-07-06 03:19:53.040000+00:00
lat                                    35.7695
lon                                -117.599333
depth                                   8000.0
mag                                        7.1
Name: 726, dtype: object


<IPython.core.display.Javascript object>